In [3]:
import util as ut
import constantes as cons
from backtesting import Backtest, Strategy
import numpy as np
symbol =  str.upper('cfx'+'USDT')

data = ut.estrategia(symbol,path='modelos/lstm')

## BACKTESTING
class Predictor2(Strategy):
    def init(self):
        pass
   
    def next(self):
        if not self.position:
            if self.data.signal[-1] ==1:
                self.buy(size=100, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
            elif self.data.signal[-1] ==-1:
                self.sell(size=100, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
        else:
            pass

bt = Backtest(data, Predictor2, cash=1000, commission=.002, exclusive_orders=True)

output = bt.run()
bt.plot()
output



C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2023-06-04 08:30:00
End                       2023-06-08 12:00:00
Duration                      4 days 03:30:00
Exposure Time [%]                        89.5
Equity Final [$]                  1001.768694
Equity Peak [$]                   1002.608694
Return [%]                           0.176869
Buy & Hold Return [%]              -16.214233
Return (Ann.) [%]                    6.462517
Volatility (Ann.) [%]                2.188807
Sharpe Ratio                         2.952529
Sortino Ratio                        5.511691
Calmar Ratio                        24.176776
Max. Drawdown [%]                   -0.267303
Avg. Drawdown [%]                   -0.045338
Max. Drawdown Duration        2 days 15:00:00
Avg. Drawdown Duration        0 days 09:17:00
# Trades                                   12
Win Rate [%]                        91.666667
Best Trade [%]                       2.333982
Worst Trade [%]                     -3.219028
Avg. Trade [%]                    